In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
from numpy import random

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/satya/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
tips = pd.read_csv("/Users/satya/Yelp Tips with Metro.csv")
tips_df = pd.DataFrame(tips)
missing_tips = tips_df['text'].isna().sum()
# drop 12 rows with no tips
tips_df = tips_df.dropna(subset = ['text'])

# List of metro areas to filter out
# excluding NYC and NJ because there's only a few restaurants in this dataset from that metro area
excluded_metros = ['Truckee-Grass Valley, CA', 'Reading, PA', 'Trenton-Princeton, NJ', 'Unknown Metro Area', 'Atlantic City-Hammonton, NJ', 
'New York-Newark-Jersey City, NY-NJ' ]

tips_with_metro = tips_df[~tips_df['metro'].isin(excluded_metros)]
print("Major City Tips:", tips_with_metro.shape)

Major City Tips: (703543, 20)


In [ ]:
tips_with_metro['sentences'] = tips_with_metro['text'].str.replace(r'[.!?]', '.', regex=True).str.split('.')
tips_with_metro = tips_with_metro.explode('sentences')

/var/folders/00/hsjhd04n2kjgfbfc370y7wzr0000gn/T/ipykernel_21034/3142755887.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tips_with_metro['sentences'] = tips_with_metro['text'].str.replace(r'[.!?]', '.', regex=True).str.split('.')


In [ ]:
stlouis = tips_with_metro[tips_with_metro['metro'] == 'St. Louis, MO-IL']
stlouis = stlouis.groupby('business_id').filter(lambda x: x['text'].nunique() > 10)
stlouis['city'] = stlouis['city'].str.strip()
stlouis_only = stlouis[(stlouis['city'] == 'St. Louis') | (stlouis['city'] == 'Saint Louis')]
city_review_counts = stlouis_only.groupby(['business_id', 'name', 'address', 'city'])['text'].nunique().sort_values(ascending=False)
display(city_review_counts)

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import re
import time

# Set up Selenium driver
driver = webdriver.Safari()
wait = WebDriverWait(driver, 10)

# Define regex pattern for food code numbers
food_code_pattern = r"\d{1,2}-\d{1,3}"

df = pd.DataFrame(columns=["Name", "Address", "Critical Violations", "Non-Critical Violations", "Inspection Date", "Letter Grade", "Violation Details"])

stlouis = stlouis.drop_duplicates(subset=['name', 'address'])

for index, row in stlouis.iterrows():
    name = row['name']
    address = row['address']
    
    try:
        driver.get("https://www.healthspace.com/StLouis-MO/")
        time.sleep(4)

        # Switch to the 'bottom' frame and interact with the "Food facility inspections" button
        try:
            driver.switch_to.frame("bottom")
            element = wait.until(EC.visibility_of_element_located((By.LINK_TEXT, "Food facility inspections")))
            element.click()
            driver.switch_to.default_content()
            time.sleep(4)
        except Exception as e:
            print(f"Error interacting with Food Facility button: {e}")
            continue
        
        # Enter the search details
        try:
            driver.switch_to.frame("bottom")
            driver.switch_to.frame("search")
            time.sleep(4)
            search_box = wait.until(EC.presence_of_element_located((By.NAME, "Query")))
            search_box.send_keys(address)
            go_button = driver.find_element(By.XPATH, "//input[@value='Go']")
            go_button.click()
            driver.switch_to.default_content()
        except Exception as e:
            print(f"Error interacting with the search field: {e}")
            continue

        # Locate and click the first inspection link
        try:
            driver.switch_to.frame("bottom")
            driver.switch_to.frame("body")
            time.sleep(4)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Wait for page content to load
            first_row_link = wait.until(EC.presence_of_element_located((By.XPATH, "//body//table//tr[1]//a[contains(@href, 'Food-FacilityHistory')]")))
            first_row_link.click()
            driver.switch_to.default_content()
            time.sleep(4)

        except NoSuchElementException:
            print(f"Inspection link not found for {name}. Skipping.")
            df = pd.concat([df, pd.DataFrame({
                "Name": [name],
                "Address": [address],
                "Critical Violations": ["Not Found"],
                "Non-Critical Violations": ["Not Found"],
                "Inspection Date": ["Not Found"],
                "Letter Grade": ["Not Found"],
                "Violation Details": ["Not Found"]
            })], ignore_index=True)
            continue
        except TimeoutException:
            print(f"Inspection link not found for {name}. Skipping.")
            df = pd.concat([df, pd.DataFrame({
                "Name": [name],
                "Address": [address],
                "Critical Violations": ["Not Found"],
                "Non-Critical Violations": ["Not Found"],
                "Inspection Date": ["Not Found"],
                "Letter Grade": ["Not Found"],
                "Violation Details": ["Not Found"]
            })], ignore_index=True)
            continue

        # Locate inspection details
        driver.switch_to.frame("bottom")
        driver.switch_to.frame("body")
        time.sleep(4)
        inspection_link = wait.until(EC.presence_of_element_located((By.XPATH, "//body//table//tr[1]//a[contains(@href, 'Food-InspectionDetails')]")))
        inspection_link.click()
        time.sleep(4)

        try:
            critical_violations_value = wait.until(EC.presence_of_element_located(
                (By.XPATH, "//td[contains(text(), 'Number of critical violations:')]/following-sibling::td")
            )).text.strip()
            non_critical_violations_value = wait.until(EC.presence_of_element_located(
                (By.XPATH, "//td[contains(text(), 'Number of non-critical violations:')]/following-sibling::td")
            )).text.strip()
            inspection_date_value = wait.until(EC.presence_of_element_located(
                (By.XPATH, "//td[contains(text(), 'Inspection date:')]/following-sibling::td")
            )).text.strip()
            letter_grade_value = wait.until(EC.presence_of_element_located(
                (By.XPATH, "//td[contains(text(), 'Letter Grade:')]/following-sibling::td")
            )).text.strip()
        except Exception as e:
            print(f"Error extracting data: {e}")
            critical_violations_value = non_critical_violations_value = inspection_date_value = letter_grade_value = "Not Found"

        try:
            # Locate all rows in the violations table
            violation_rows = driver.find_elements(By.XPATH, "//tbody/tr/td")
            
            violation_details_value = ""
            
            for row in violation_rows:
                violation_text = row.text.strip()
                
                # Search for food code numbers in the violation text using regex
                matches = re.findall(food_code_pattern, violation_text)
                if matches:
                    # Append the matching text and food codes to the violation details
                    violation_details_value += f"Violation: {violation_text}\nFood Codes Found: {', '.join(matches)}\n"
            
            if not violation_details_value:
                violation_details_value = "No violation details found"

        except Exception as e:
            print(f"Error extracting violation details: {e}")
            violation_details_value = "Not Found"

        restaurant_data = {
            "Name": name,
            "Address": address,
            "Critical Violations": critical_violations_value,
            "Non-Critical Violations": non_critical_violations_value,
            "Inspection Date": inspection_date_value,
            "Letter Grade": letter_grade_value,
            "Violation Details": violation_details_value
        }

        df = pd.concat([df, pd.DataFrame([restaurant_data])], ignore_index=True)

    except Exception as e:
        print(f"Error in parsing {name}: {e}")

df.to_csv("restaurant_health_inspections.csv", index=False)

driver.quit()


In [16]:
display(df)

,Name,Address,Critical Violations,Non-Critical Violations,Inspection Date,Letter Grade,Violation Details
0,Tony's Restaurant & 3rd Street Cafe,312 Piasa St,Not Found,Not Found,Not Found,Not Found,Not Found
1,Budweiser Brewery Experience,1200 Lynch St,Not Found,Not Found,Not Found,Not Found,Not Found
2,McDonald's,1919 S Jefferson,0,0,28-March-2024,A,No violation details found
3,Carmine's Steakhouse,20 S 4th St,0,0,19-July-2024,A,No violation details found
4,New India's Kitchen,14222 Manchester Rd,Not Found,Not Found,Not Found,Not Found,Not Found
...,...,...,...,...,...,...,...
1225,Starbucks,3884 State Route 141,Not Found,Not Found,Not Found,Not Found,Not Found
1226,Corner 17,6623 Delmar Blvd,Not Found,Not Found,Not Found,Not Found,Not Found
1227,Yaquis,2728 Cherokee St,0,1,07-March-2024,A,Violation: 4-602.13 / Nonfood-Contact Surfaces...
1228,Tower Grove Park,4256 Magnolia Ave,0,0,12-August-2023,A,No violation details found


In [17]:
df.to_csv("restaurant_health_inspections.csv", index=False)

In [18]:
health_inspections = pd.read_csv("restaurant_health_inspections.csv")
health_inspections = pd.DataFrame(health_inspections)
# remove all occurances of "Not Found"
health_inspections = health_inspections.replace('Not Found', np.nan)
health_inspections = health_inspections.dropna()

In [19]:
health_inspections.shape

(220, 7)

In [20]:
stlouis_only['sentences'] = stlouis_only['text'].str.replace(r'[.!?]', '.', regex=True).str.split('.')
stlouis_only = stlouis_only.explode('sentences')

In [21]:
sia = SentimentIntensityAnalyzer()
stlouis_only['pos'] = stlouis_only['sentences'].apply(lambda x: sia.polarity_scores(x)['pos'])
stlouis_only['neg'] = stlouis_only['sentences'].apply(lambda x: sia.polarity_scores(x)['neg'])
stlouis_only['neutral'] = stlouis_only['sentences'].apply(lambda x: sia.polarity_scores(x)['neu'])
stlouis_only['compound'] = stlouis_only['sentences'].apply(lambda x: sia.polarity_scores(x)['compound'])

In [22]:
display(stlouis_only)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,text,date,compliment_count,geometry,metro,sentences,pos,neg,neutral,compound
620,sB45WFgysT617bKWP_WJwA,Budweiser Brewery Experience,1200 Lynch St,Saint Louis,MO,63118,38.600197,-90.213538,4.5,605,...,the biergarten has free wifi!,2016-02-05 20:58:56,0,POINT (-90.2135377 38.6001971),"St. Louis, MO-IL",the biergarten has free wifi,0.452,0.000,0.548,0.5106
620,sB45WFgysT617bKWP_WJwA,Budweiser Brewery Experience,1200 Lynch St,Saint Louis,MO,63118,38.600197,-90.213538,4.5,605,...,the biergarten has free wifi!,2016-02-05 20:58:56,0,POINT (-90.2135377 38.6001971),"St. Louis, MO-IL",,0.000,0.000,0.000,0.0000
621,sB45WFgysT617bKWP_WJwA,Budweiser Brewery Experience,1200 Lynch St,Saint Louis,MO,63118,38.600197,-90.213538,4.5,605,...,Where better to burn a couple of hours?,2011-10-08 16:53:56,0,POINT (-90.2135377 38.6001971),"St. Louis, MO-IL",Where better to burn a couple of hours,0.326,0.000,0.674,0.4404
621,sB45WFgysT617bKWP_WJwA,Budweiser Brewery Experience,1200 Lynch St,Saint Louis,MO,63118,38.600197,-90.213538,4.5,605,...,Where better to burn a couple of hours?,2011-10-08 16:53:56,0,POINT (-90.2135377 38.6001971),"St. Louis, MO-IL",,0.000,0.000,0.000,0.0000
622,sB45WFgysT617bKWP_WJwA,Budweiser Brewery Experience,1200 Lynch St,Saint Louis,MO,63118,38.600197,-90.213538,4.5,605,...,Beer School only $10.,2012-11-15 21:08:54,0,POINT (-90.2135377 38.6001971),"St. Louis, MO-IL",Beer School only $10,0.000,0.000,1.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718464,WXNag-tmudrFHFlfNLogWg,Whole Foods Market,4577 W Pine Blvd,St. Louis,MO,63108,38.641988,-90.261655,4.0,71,...,Needs to get their act together. I've waited a...,2016-06-21 00:41:16,0,POINT (-90.26165498 38.64198815),"St. Louis, MO-IL",Lines are sometimes crazy - they haven't figu...,0.000,0.391,0.609,-0.6973
718464,WXNag-tmudrFHFlfNLogWg,Whole Foods Market,4577 W Pine Blvd,St. Louis,MO,63108,38.641988,-90.261655,4.0,71,...,Needs to get their act together. I've waited a...,2016-06-21 00:41:16,0,POINT (-90.26165498 38.64198815),"St. Louis, MO-IL",Seafood department in particular is still tot...,0.000,0.000,1.000,0.0000
718464,WXNag-tmudrFHFlfNLogWg,Whole Foods Market,4577 W Pine Blvd,St. Louis,MO,63108,38.641988,-90.261655,4.0,71,...,Needs to get their act together. I've waited a...,2016-06-21 00:41:16,0,POINT (-90.26165498 38.64198815),"St. Louis, MO-IL",Twice I've bought oysters (Rock Point) that h...,0.000,0.280,0.720,-0.5423
718464,WXNag-tmudrFHFlfNLogWg,Whole Foods Market,4577 W Pine Blvd,St. Louis,MO,63108,38.641988,-90.261655,4.0,71,...,Needs to get their act together. I've waited a...,2016-06-21 00:41:16,0,POINT (-90.26165498 38.64198815),"St. Louis, MO-IL",Cheese department is great but otherwise it f...,0.221,0.000,0.779,0.3716


In [23]:
display(health_inspections)

,Name,Address,Critical Violations,Non-Critical Violations,Inspection Date,Letter Grade,Violation Details
2,McDonald's,1919 S Jefferson,0,0,28-March-2024,A,No violation details found
3,Carmine's Steakhouse,20 S 4th St,0,0,19-July-2024,A,No violation details found
6,Soco's Gyros,5530 S Lindbergh Blvd,0,0,29-September-2023,A,No violation details found
12,Grace Meat + Three,4270 Manchester,0,0,17-May-2024,A,No violation details found
17,Kampai Sushi Bar,4949 W Pine Blvd,0,6,14-May-2024,A,"Violation: 4-601.11(c) / Equipment, Food-Conta..."
...,...,...,...,...,...,...,...
1196,Manchester Public House,6655 Manchester Ave,0,2,10-April-2024,A,Violation: 4-101.11 b-e / Characteristics. ...
1209,Nara Cafe & Hookah Lounge,1326 Washington Ave,0,5,01-February-2024,A,"Violation: 4-501.14 / Warewashing Equipment, C..."
1216,Southern,3108 Olive St,0,0,05-April-2024,A,No violation details found
1227,Yaquis,2728 Cherokee St,0,1,07-March-2024,A,Violation: 4-602.13 / Nonfood-Contact Surfaces...


In [24]:
# creating key to match 
stlouis_only['name_address'] = stlouis_only['name'] + ' ' + stlouis_only['address']
health_inspections['name_address'] = health_inspections['Name'] + ' ' + health_inspections['Address']

In [34]:

rating_and_score = pd.merge(health_inspections, stlouis_only, on="name_address", how="left")


In [35]:
rating_and_score_view = rating_and_score.groupby(['business_id', 'name', 'address', 'city'])['text'].nunique().sort_values(ascending=False)
print(rating_and_score_view)

business_id             name                     address              city       
nRKndeZLQ3eDL10UMwS2rQ  Ted Drewes               6726 Chippewa St     Saint Louis    325
NHb6QTrJOnyIj-Zd0fYzFg  Bogart's Smokehouse      1627 S 9th St        Saint Louis    249
FryXEasH74MQDnj5Bvz3QA  The Mud House            2101 Cherokee St     Saint Louis    236
F4gvookXrKsBNCvK_xEt1g  The Schlafly Tap Room    2100 Locust St       Saint Louis    176
jQBPO3rYkNwIaOdQS5ktgQ  The Fountain On Locust   3037 Locust St       Saint Louis    166
                                                                                    ... 
LLpsnCJFEyj55aA8ANLmog  Cork n' Slice            4501 Maryland Ave    Saint Louis     11
ZoSDp6Ycoy9H2spud3PYqQ  Turn Restaurant          3224 Locust St       St. Louis       11
fsKjL-yPx-wyZXXoti_FDw  Manchester Public House  6655 Manchester Ave  Saint Louis     11
hRPH-pAQmxSiyyxTfljusA  The Hideaway             5900 Arsenal St      Saint Louis     11
ErraEP2corkgkzmh62t45A  Red 

In [ ]:
rating_and_score.to_csv("Rating_and_Score_St_Louis.csv", index = False)

In [ ]:
rating_and_score = pd.read_csv('/Users/satya/Documents/MSA 24-25/Fall 1/Text Analytics/Yelp Final Project/Rating_and_Score_St_Louis.csv')

# Group by 'business_id' and calculate the metrics
sentence_sentiments = rating_and_score.groupby('business_id').agg(
    total_sentences=('text', 'count'),  # Total reviews for this restaurant
    total_tips=('text', 'nunique'),    # Count of unique reviews/tips
    sum_positive=('pos', 'sum'),       # Sum of positive sentiment
    sum_negative=('neg', 'sum'),       # Sum of negative sentiment
    sum_neutral=('neutral', 'sum'),    # Sum of neutral sentiment
    sentiment_variance=('compound', 'var')  # Variance of compound sentiment scores
).reset_index()

# Calculate total sentiment score
sentence_sentiments['total_sentiment'] = (
    sentence_sentiments['sum_positive'] +
    sentence_sentiments['sum_negative'] +
    sentence_sentiments['sum_neutral']
)

# Calculate percentage positive and negative sentiment
sentence_sentiments['percent_positive'] = (
    (sentence_sentiments['sum_positive'] / sentence_sentiments['total_sentiment']) * 100
)
sentence_sentiments['percent_negative'] = (
    (sentence_sentiments['sum_negative'] / sentence_sentiments['total_sentiment']) * 100
)
sentence_sentiments['percent_neutral'] = (
    (sentence_sentiments['sum_neutral'] / sentence_sentiments['total_sentiment']) * 100
)

result =  rating_and_score.merge(sentence_sentiments, on='business_id')
display(result)


,Name,Address,Critical Violations,Non-Critical Violations,Inspection Date,Letter Grade,Violation Details,name_address,business_id,name,...,total_sentences,total_tips,sum_positive,sum_negative,sum_neutral,sentiment_variance,total_sentiment,percent_positive,percent_negative,percent_neutral
0,McDonald's,1919 S Jefferson,0,0,28-Mar-24,A,No violation details found,McDonald's 1919 S Jefferson,yM8LlTInbQH4FwWC97lz6w,McDonald's,...,58,20,3.426,7.144,31.430,0.096236,42.0,8.157143,17.009524,74.833333
1,McDonald's,1919 S Jefferson,0,0,28-Mar-24,A,No violation details found,McDonald's 1919 S Jefferson,yM8LlTInbQH4FwWC97lz6w,McDonald's,...,58,20,3.426,7.144,31.430,0.096236,42.0,8.157143,17.009524,74.833333
2,McDonald's,1919 S Jefferson,0,0,28-Mar-24,A,No violation details found,McDonald's 1919 S Jefferson,yM8LlTInbQH4FwWC97lz6w,McDonald's,...,58,20,3.426,7.144,31.430,0.096236,42.0,8.157143,17.009524,74.833333
3,McDonald's,1919 S Jefferson,0,0,28-Mar-24,A,No violation details found,McDonald's 1919 S Jefferson,yM8LlTInbQH4FwWC97lz6w,McDonald's,...,58,20,3.426,7.144,31.430,0.096236,42.0,8.157143,17.009524,74.833333
4,McDonald's,1919 S Jefferson,0,0,28-Mar-24,A,No violation details found,McDonald's 1919 S Jefferson,yM8LlTInbQH4FwWC97lz6w,McDonald's,...,58,20,3.426,7.144,31.430,0.096236,42.0,8.157143,17.009524,74.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21770,Tower Grove Park,4256 Magnolia Ave,0,0,12-Aug-23,A,No violation details found,Tower Grove Park 4256 Magnolia Ave,E359ITHKa9CVe9cTOofZUQ,Tower Grove Park,...,175,72,23.149,2.754,71.097,0.079715,97.0,23.864948,2.839175,73.295876
21771,Tower Grove Park,4256 Magnolia Ave,0,0,12-Aug-23,A,No violation details found,Tower Grove Park 4256 Magnolia Ave,E359ITHKa9CVe9cTOofZUQ,Tower Grove Park,...,175,72,23.149,2.754,71.097,0.079715,97.0,23.864948,2.839175,73.295876
21772,Tower Grove Park,4256 Magnolia Ave,0,0,12-Aug-23,A,No violation details found,Tower Grove Park 4256 Magnolia Ave,E359ITHKa9CVe9cTOofZUQ,Tower Grove Park,...,175,72,23.149,2.754,71.097,0.079715,97.0,23.864948,2.839175,73.295876
21773,Tower Grove Park,4256 Magnolia Ave,0,0,12-Aug-23,A,No violation details found,Tower Grove Park 4256 Magnolia Ave,E359ITHKa9CVe9cTOofZUQ,Tower Grove Park,...,175,72,23.149,2.754,71.097,0.079715,97.0,23.864948,2.839175,73.295876


In [43]:
result.to_csv("Grouped_St_Louis.csv", index = False)